<h3>신경망 모델 구성하기</h3>
  
신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다. torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성요소를 제공합니다. Pytorch의 모든 모듈은 nn.module의 하위클래스입니다. 신경망은 다른모듈 (계층;layer)로 구성된 모듈임.
이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device
)

cpu


<h4>클래스 정의하기</h4>
신경망 모델을 nn.moduled의 하위클래스로 정의하고, __init__에서 신경망 계층들을 초기화한다. nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력데이터에 대한 연산들을 구현한다.


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward(self,x):
        x = self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits



In [3]:
#NeuralNetwork의 인스턴스를 생성하고 이를 device로 이동한 뒤, 구조를 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연산들 과 함께 모델의 forward 를 실행합니다. model.forward() 를 직접 호출하지 마세요!

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10-차원 텐서가 반환됩니다. 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [4]:
X = torch.rand(1,28,28,device=device)
logits=model(X)
pred_probab=nn.Softmax(dim=1)(logits)#softmax에 통과시켜 예측확률을 얻음
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")


Predicted class: tensor([7])


<h4>모델 계층(Layer)</h4>
  
FashionMNIST 모델의 계층들을 살펴보자. 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤일이 발생하는지 알아보자.


In [5]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


<h4>nn.Flatten</h4>
  
nn.Flatten 계층을 초기화하여 각 28x28의 2D이미지를 784 픽셀 값을 갖는 연속된 배열로 변환한다. (dim = 0의 미니배치 차원은 유지된다.)


In [6]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


<h4>nn.Linear</h4>
  
선형계층은 저장된 가중치와 편향을 사용하여 입력에 선형변환을 적용하는 모듈

In [7]:
layer1=nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


<h4>nn.ReLU</h4>
  
비선형 활성화는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만든다. 비선형 활성화는 선형변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.
  
이 모델에서는 nn.ReLU를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수 있다.

In [8]:
print(f"Before Relu: {hidden1} \n\n")
hidden1=nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before Relu: tensor([[ 0.2123, -0.0275, -0.0568, -0.4576, -0.1948, -0.2389, -0.0688, -0.0324,
          0.1677,  0.6561, -0.3389, -0.2413, -0.1363, -0.6253, -0.0991, -0.1113,
          0.3915,  0.0798, -0.3151,  0.6526],
        [ 0.3226,  0.2484,  0.2536,  0.0179, -0.3702,  0.0438,  0.2113, -0.0741,
          0.0993,  0.4568, -0.1104, -0.4079, -0.0805, -0.3136,  0.1521, -0.2949,
          0.5390,  0.1068, -0.1769,  0.3526],
        [ 0.2549,  0.0892, -0.0128, -0.3595, -0.2989, -0.0173,  0.4845,  0.0323,
          0.0692,  0.6072, -0.1412, -0.1017,  0.1388, -0.2360, -0.0405, -0.2122,
          0.1216,  0.4521,  0.0138,  0.6410]], grad_fn=<AddmmBackward0>) 


After ReLU: tensor([[0.2123, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1677,
         0.6561, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3915, 0.0798,
         0.0000, 0.6526],
        [0.3226, 0.2484, 0.2536, 0.0179, 0.0000, 0.0438, 0.2113, 0.0000, 0.0993,
         0.4568, 0.0000, 0.0000, 0.0000, 0.0000, 0.1

<h4>nn.Sequential</h4>

nn.Sequential은 순서를 갖는 모듈의 컨테이너이다. 데이터는 정의된것과 같은 순서로 모든 모듈들을 통해 전달된다. 순차 컨테이너를 사용하여 아래의 seq_modeules와 같은 신경망을 빠르게 만들 수 있다.

In [9]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image=torch.rand(3,28,28)
logits = seq_modules(input_image)


<h4>nn.Softmax</h4>
  
신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty,infty] 범위의 원시 값(raw value)인) logit를 반환한다. logits는 모델의 각 분류에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정된다. dim매개변수는 값의 합이 1이 되는 차원을 나타낸다.



In [10]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

<h4>모델 매개변수</h4>
  
신경망 내부의 많은 계층들은 매개변수화된다. 즉, 학습중에 최적화되는 가중치와 편향과 연관지어진다. nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 및 named_parameters()메소드로 모든 매개변수에 접근할 수 있게된다.

In [11]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0323,  0.0106,  0.0196,  ...,  0.0264, -0.0314, -0.0140],
        [-0.0035, -0.0299,  0.0155,  ..., -0.0254, -0.0102, -0.0143]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0106, -0.0182], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0397, -0.0076, -0.0401,  ...,  0.0272,  0.0412,  0.0369],
        [-0.0300, -0.0289, -0.0106,  ..., -0.0421, -0.0033, -0.0392]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 